In [1]:
from genetic_algorithm import *
from numpy.random import default_rng
import matplotlib.pyplot as plt

In [2]:

# ## hexadecimal generator

nr_of_input = 3
nr_of_actions = 5
nr_of_inner = 3
nr_of_genes = 16
nr_individuals = 4

individuals = generate_initial_genomes_for_population(nr_individuals, nr_of_genes, nr_of_input, nr_of_actions, nr_of_inner)

## world size
world_size_x = 20
world_size_y = 20

rng = default_rng()
x = rng.choice(world_size_x, size=nr_individuals, replace=False)
y = rng.choice(world_size_y, size=nr_individuals, replace=False)

# ## initial brain and position generator

result = calculate_individual_output_weights(individuals)

for indiv in result:
    result[indiv]['position'] = [[x[indiv], y[indiv]]]

print(result)



{0: {'out4': 0.996, 'out3': 0.001, 'out2': 0.762, 'out1': 0.996, 'position': [[3, 18]]}, 1: {'out4': 0.0, 'out2': 0.999, 'out3': 0.0, 'out1': 0.957, 'position': [[19, 4]]}, 2: {'out0': 0.995, 'position': [[10, 1]]}, 3: {'out0': 0.988, 'out2': 0.006, 'out3': 0.997, 'position': [[7, 17]]}}


In [3]:
result = {0: {'out3': 1.0, 'out2': 0.087, 'out0': 0.995, 'out4': 0.992, 'out1': 0.069, 'position': [[14, 4]]},
          1: {'out1': 0.004, 'out2': 0.318, 'out4': 0.966, 'out3': 0.005, 'out0': 0.997, 'position': [[4, 5]]},
          2: {'out4': 1.0, 'out0': 0.048, 'out2': 0.88, 'out1': 0.732, 'out3': 0.831, 'position': [[3, 3]]},
          3: {'out2': 0.536, 'out1': 1.0, 'out4': 0.067, 'position': [[17, 15]]}}

In [8]:
k = 5
n = 0
while k>n:  
    for indiv in result:
        x, y = result[indiv]['position'][-1][0], result[indiv]['position'][-1][1]

        position_list = []
        for out in result[indiv]:
            if 'out' in out:
                new_pos = move(out, result[indiv][out])

                position_list.append(new_pos)
        
        position_list = list(map(sum, zip(*position_list)))
        position_list = make_smaller_(position_list)
        position_list = list(map(sum, zip(*[[x, y]] + [position_list])))
        
#         for j in range(len(position_list)):
#             for i_nr, i in enumerate(position_list[j+1:], start=j+1):
#                 if position_list[j] == i:
#                     position_list[i_nr] = result[indiv]['position'][-1]
                    
        position_list[0] = normalize_position_if_outside_world(position_list[0], world_size_x)
        position_list[1] = normalize_position_if_outside_world(position_list[1], world_size_y)
#         print('norm', position_list)
        result[indiv]['position'].append(position_list)
    
    last_pos_list = [result[obj]['position'][-1] for obj in result]
   
    for obj in range(len(last_pos_list)):
        for i_nr, i in enumerate(last_pos_list[obj+1:], start=obj+1):
            if last_pos_list[obj] == i:
                result[i_nr]['position'][-1] = result[i_nr]['position'][-2]     
    n += 1
#     print()        
# result

In [13]:
x1,y1,x2,y2 = 3,0,3,1
dx = x2-x1
dy = y2-y1
x3,y3 = x2+dx,y2+dy
# print(x3,y3)
print('source', dx,dy)
for i in range(5):
    if dx != 0:
        dx += 1
    if dy != 0:
        dy += 1
    print(dx,dy)

source 0 1
0 2
0 3
0 4
0 5
0 6


In [10]:
for i in range(5):
    print(i)

0
1
2
3
4


In [5]:
a = [(3, 0),(4,1),(5,2)]

def slope_intercept(x1,y1,x2,y2):
    a = (y2 - y1) / (x2 - x1)
    b = y1 - a * x1     
    return a,b

# print(slope_intercept(3,0,2,1))

if a[-2][0] == a[-1][0]:
    x = a[-1][0]
elif a[-2][0] > a[-1][0]:
    x = a[-2][0] + 1
elif a[-2][0] < a[-1][0]:
    x = a[-2][0] - 1
# print(x)

a[-2][0]



4